In [23]:
import geopandas
import pandas as pd
from google.cloud.bigquery import Client
import subprocess
import os

In [21]:
def upload_geo_to_gbq(geo_zip_dir_name, gbq_loc):
    print("=" * 80)
    print("Unzipping geo data:\n")
    subprocess.run(["unzip", "-d", geo_zip_dir_name, geo_zip_dir_name + ".zip"])
    gdf = geopandas.read_file(geo_zip_dir_name)
    print("\nConverting geometries to GCP format")
    gdf = gdf.to_crs(epsg=4326)
    gdf["geometry"] = gdf.geometry.astype("string")
    print("\nUploading geo data to GBQ")
    gdf.to_gbq(gbq_loc + "_tmp")
    sql = f"""
        CREATE TABLE `{gbq_loc}` AS
        SELECT * EXCEPT(geometry), 
            st_geogfromtext(geometry, make_valid => TRUE) as geometry, 
            SAFE.st_geogfromtext(geometry) IS NULL AS geom_needed_fix
        FROM `{gbq_loc}_tmp`
    """
    client = Client()
    print("\nConverting geometry to GEOGRAPHY type")
    client.query(sql).result()
    client.delete_table(gbq_loc + "_tmp")
    print("\nDONE!!")

In [24]:
geo_zip_dir_name = "Wards_(December_2019)_Boundaries_EW_BFC"
gbq_loc = "yhcr-prd-phm-bia-core.CY_MYSPACE_SR.geog_test"

upload_geo_to_gbq(geo_zip_dir_name, gbq_loc)

Unzipping geo data:

Archive:  Wards_(December_2019)_Boundaries_EW_BFC.zip
  inflating: Wards_(December_2019)_Boundaries_EW_BFC/Wards_(December_2019)_Boundaries_EW_BFC.shp  
  inflating: Wards_(December_2019)_Boundaries_EW_BFC/Wards_(December_2019)_Boundaries_EW_BFC.shx  
  inflating: Wards_(December_2019)_Boundaries_EW_BFC/Wards_(December_2019)_Boundaries_EW_BFC.dbf  
 extracting: Wards_(December_2019)_Boundaries_EW_BFC/Wards_(December_2019)_Boundaries_EW_BFC.cpg  
  inflating: Wards_(December_2019)_Boundaries_EW_BFC/Wards_(December_2019)_Boundaries_EW_BFC.prj  
  inflating: Wards_(December_2019)_Boundaries_EW_BFC/Wards_(December_2019)_Boundaries_EW_BFC.xml  


/opt/conda/envs/profileEnv/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


Uploading geo data to GBQ
Converting geometry to GEOGRAPHY type
